<a href="https://www.kaggle.com/code/doauinc/osrs-bond-price-forecast?scriptVersionId=232904742" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Problem Statement
Forecast the price of OSRS bonds using time series modeling to understand market trends and volatility.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## EDA & Seasonality

In [ ]:
df = pd.read_csv("/kaggle/input/oldschool-runescape-bond-price-history/osrs_old_school_bond_history.csv", parse_dates=['Date'])

# Create a display-friendly version
display_df = df.copy()
display_df['Price (GP)'] = display_df['Price (GP)'].map('{:,}'.format)
display_df

In [ ]:
df['Price (M)'] = df['Price (GP)'] / 1_000_000  # Convert to millions

print("=== Data Overview ===")
print(f"Total records: {len(df)}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")

print("\n=== Price Statistics ===")
df['Price (M)'].describe()

In [ ]:
import plotly.graph_objects as go
from datetime import datetime

# Big game updates
events = [
    {'date': datetime(2018, 10, 30), 'name': 'Mobile Release', 'type': 'update'},
    {'date': datetime(2020, 3, 23), 'name': 'COVID Lockdown', 'type': 'event'},
    {'date': datetime(2022, 8, 24), 'name': 'Tombs of Amascut', 'type': 'update'},
    {'date': datetime(2023, 7, 26), 'name': 'Desert Treasure 2', 'type': 'update'},
    {'date': datetime(2019, 11, 14), 'name': 'Twisted League', 'type': 'league'},
    {'date': datetime(2020, 10, 28), 'name': 'Trailblazer League', 'type': 'league'},
    {'date': datetime(2022, 1, 19), 'name': 'Shattered Relics', 'type': 'league'},
    {'date': datetime(2023, 11, 15), 'name': 'Trailblazer Reloaded', 'type': 'league'},
    {'date': datetime(2024, 11, 27), 'name': 'Raging Echoes League', 'type': 'league'}
]

# Create figure
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Price (M)'],
    mode='lines',
    name='Bond Price',
    line=dict(color='#4CC9F0', width=2.5),  # Bright cyan line
    hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Price: %{y:,.2f}M<extra></extra>'
))

for event in events:
    color = '#FFD700' if event['type'] == 'league' else '#7209B7'  # Pink/purple
    dash = 'dot' if event['type'] == 'league' else 'dash'

    date = event['date']   
    closest_idx = (df['Date'] - date).abs().idxmin()
    y_loc = df.loc[closest_idx, 'Price (M)'] + 5

    fig.add_vline(
        x=date,
        line=dict(color=color, width=1.5, dash=dash),
        opacity=0.7
    )
    
    fig.add_annotation(
        x=date,
        y=y_loc,
        text=event['name'],
        showarrow=False,
        font=dict(size=14, color=color, family="Arial Black"),
        textangle=-90,
        xanchor='right',
        yanchor='middle',
        xshift=2 
    )

# Dark theme layout
fig.update_layout(
    title=dict(
        text='<b>OSRS BOND PRICE HISTORY</b>',
        y=0.95,
        x=0.5,
        font=dict(size=24, color='#FFFFFF', family="Arial Black"),
        xanchor='center'
    ),
    xaxis=dict(
        title='<b>DATE</b>',
        title_font=dict(size=14, color='#B8B8B8'),
        tickfont=dict(color='#B8B8B8'),
        gridcolor='rgba(100,100,100,0.2)',
        linecolor='#404040',
        rangeslider=dict(visible=True)
    ),
    yaxis=dict(
        title='<b>PRICE (MILLIONS GP)</b>',
        title_font=dict(size=14, color='#B8B8B8'),
        tickfont=dict(color='#B8B8B8'),
        gridcolor='rgba(100,100,100,0.2)',
        linecolor='#404040',
        tickformat=',.0f'
    ),
     hovermode='x unified',
     hoverlabel=dict(
        bgcolor='rgba(30, 30, 30, 0.8)',  # Semi-transparent dark background
        font=dict(size=24, color='#000000', family="Arial Black"),
        bordercolor='rgba(255, 255, 255, 0.2)',  # Light border
        namelength=-1  # Show full name
    ),
    height=750,
    margin=dict(t=100, b=80),
    plot_bgcolor='#121212',
    paper_bgcolor='#121212',
    legend=dict(
        font=dict(color='#FFFFFF'),
        orientation='h',
        y=1.02
    )
)

fig.update_xaxes(showgrid=True, gridwidth=0.5)
fig.update_yaxes(showgrid=True, gridwidth=0.5)
fig.show()

In [ ]:
# Convert and prepare data
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Calculate monthly percentage changes
monthly_data = df.groupby(['Year', 'Month']).agg({
    'Price (M)': ['first', 'last'],
    'Date': 'count'
}).reset_index()

monthly_data.columns = ['Year', 'Month', 'Start Price', 'End Price', 'Days']
monthly_data['Pct Change'] = ((monthly_data['End Price'] - monthly_data['Start Price']) / 
                             monthly_data['Start Price']) * 100  # Percentage change

# Analyze by month across all years
monthly_trends = monthly_data.groupby('Month').agg({
    'Pct Change': ['mean', 'median', 'max']
}).reset_index()

monthly_trends.columns = ['Month', 'Avg Pct Change', 'Median Pct Change', 'Max Pct Change']

# Sort by strongest months
strongest_months = monthly_trends.sort_values('Avg Pct Change', ascending=False)

# Visualization
plt.figure(figsize=(12, 6))
bars = plt.bar(strongest_months['Month'], strongest_months['Avg Pct Change'],
               color=np.where(strongest_months['Avg Pct Change'] > 0, 'green', 'red'))

plt.title('Average Monthly Bond Price Percentage Changes (All Years)')
plt.xlabel('Month')
plt.ylabel('Average % Change')
plt.xticks(range(1,13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.grid(axis='y')

# Annotate bars
for bar in bars:
    height = bar.get_height()
    offset = 0.1  # Distance from bar end
    y_pos = height + (offset if height >=0 else -offset)

    plt.text(bar.get_x() + bar.get_width()/2.,
             y_pos,
             f'{height:+.1f}%',
             ha='center',
             va='bottom' if height >=0 else 'top',
             fontsize=10)

plt.axhline(0, color='black', linestyle='--')
plt.show()


print("\n=== Strongest Months for Bond Prices (% Change) ===")
strongest_months['Month'] = strongest_months['Month'].map(lambda m: ['Jan','Feb','Mar','Apr','May','Jun',
                                                     'Jul','Aug','Sep','Oct','Nov','Dec'][m-1])
print(strongest_months.to_string(float_format=lambda x: f'{x:.1f}%' if isinstance(x, float) else str(x), 
                         index=False))


print("\n=== Top 10 Individual Monthly Gains ===")
top_gains = monthly_data.nlargest(10, 'Pct Change')[['Year', 'Month', 'Pct Change']].copy()
top_gains['Month'] = top_gains['Month'].map(lambda m: ['Jan','Feb','Mar','Apr','May','Jun',
                                                     'Jul','Aug','Sep','Oct','Nov','Dec'][m-1])
print(top_gains.to_string(float_format=lambda x: f'{x:.1f}%' if isinstance(x, float) else str(x), 
                         index=False))

In [ ]:
import seaborn as sns

# Create Year-Month grid of percentage changes
heatmap_data = monthly_data.pivot_table(index='Year', 
                                      columns='Month', 
                                      values='Pct Change')

plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, 
            cmap='RdYlGn', 
            center=0,
            annot=True, 
            fmt=".1f",
            linewidths=0.5,
            cbar_kws={'label': 'Price Change (%)'})
plt.title('OSRS Bond Price Changes by Year and Month')
plt.xlabel('Month')
plt.ylabel('Year')
plt.show()

In [ ]:
## average yearly trend ##
df['Year'] = df['Date'].dt.year

# Get first and last price of each year
yearly_prices = df.groupby('Year')['Price (GP)'].agg(['first', 'last'])

# Calculate YoY percentage change
yearly_prices['YoY Change'] = ((yearly_prices['last'] - yearly_prices['first']) / yearly_prices['first']) * 100
yearly_changes = yearly_prices.reset_index()[['Year', 'YoY Change']]
yearly_changes['YoY Change'] = yearly_changes['YoY Change'].round(1)

plt.figure(figsize=(12, 6))
bars = plt.bar(yearly_changes['Year'], yearly_changes['YoY Change'],
               color=np.where(yearly_changes['YoY Change'] >= 0, 'green', 'red'))

# Annotate bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., 
             height + (1 if height >=0 else -1),
             f'{height:.1f}%',
             ha='center', 
             va='bottom' if height >=0 else 'top')

plt.title('OSRS Bond Price Yearly Percentage Changes')
plt.xlabel('Year')
plt.ylabel('Yearly Change (%)')
plt.axhline(0, color='black', linestyle='--')
plt.grid(axis='y', alpha=0.3)
plt.show()

print(f"Average Yearly Growth: {yearly_changes['YoY Change'].mean():.1f}%")

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Set index to datetime just in case
df_vis = df.set_index('Date')

# Apply seasonal decomposition
result = seasonal_decompose(df_vis['Price (M)'], model='multiplicative', period=365)

result.plot()
plt.suptitle('Seasonal Decomposition of Bond Price', fontsize=16)
plt.tight_layout()
plt.show()

#### This decomposition breaks the original time series into trend, seasonal, and residual components. The clear upward trend and annual patterns suggest the presence of long-term growth and recurring seasonality, as expected.
#### 

## Modeling Approach

#### Prophet is designed for time series data with strong seasonality and trend—just like OSRS bond prices. It supports custom seasonalities (weekly, monthly, yearly) and changepoint detection, making it well-suited to model these price fluctuations.

In [ ]:
from prophet import Prophet
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from sklearn.metrics import mean_squared_error, mean_absolute_error
from prophet.diagnostics import cross_validation, performance_metrics

import warnings
import logging
import cmdstanpy

warnings.filterwarnings("ignore")
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('fbprophet').setLevel(logging.WARNING)

plt.style.use('ggplot')
plt.style.use('fivethirtyeight')

def mape(y_true, y_pred): # Mean Absolute Percentage Error
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# The bond price shows exponential growth over time. Applying a logarithmic transformation helps stabilize the variance
# and improve the model’s ability to capture relative changes.
df_prophet = df.reset_index().rename(columns={'Date': 'ds', 'Price (M)': 'y'})
df_prophet['y_log'] = np.log(df_prophet['y'])  

train_size = int(len(df_prophet) * 0.90)
train_df = df_prophet.iloc[:train_size]
test_df = df_prophet.iloc[train_size:]

train_df = train_df[['ds', 'y_log']].rename(columns={'y_log': 'y'}) 
test_df = test_df[['ds', 'y_log']].rename(columns={'y_log': 'y'}) 

best_model = Prophet()

best_model.fit(train_df)
forecast_train = best_model.predict(train_df[['ds']])
forecast_test = best_model.predict(test_df[['ds']])

forecast_train['yhat_original'] = np.exp(forecast_train['yhat'])
forecast_test['yhat_original'] = np.exp(forecast_test['yhat'])

train_rmse = np.sqrt(mean_squared_error(df_prophet['y'][:train_size], forecast_train['yhat_original']))
test_rmse = np.sqrt(mean_squared_error(df_prophet['y'][train_size:], forecast_test['yhat_original']))
train_mape = mape(df_prophet['y'][:train_size], forecast_train['yhat_original'])
test_mape = mape(df_prophet['y'].iloc[train_size:].values, forecast_test['yhat_original'])

plt.figure(figsize=(14, 7))
plt.plot(df_prophet['ds'], df_prophet['y'], 'b-', label='Actual Price', linewidth=1.5, alpha=0.6)
plt.plot(train_df['ds'], forecast_train['yhat_original'], 'r-', label='Predicted (Train)', linewidth=1, alpha=0.8)
plt.plot(test_df['ds'], forecast_test['yhat_original'], 'm-', label='Predicted (Test)', linewidth=1)

plt.suptitle(f'Train RMSE: {train_rmse:.2f}, Test RMSE: {test_rmse:.2f}\n Train MAPE: {train_mape:.2f}, Test MAPE: {test_mape:.2f}', fontsize=14, y=1.02)
plt.fill_between(
    forecast_test['ds'],
    np.exp(forecast_test['yhat_lower']),
    np.exp(forecast_test['yhat_upper']),
    color='pink', alpha=0.6, label='Uncertainty (Test)'
)

plt.legend(loc='upper left')
plt.title('Prophet Forecast for OSRS Bond prices')
plt.xlabel('Date')
plt.ylabel('Price (M)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
df_cv = cross_validation(
    model=best_model,
    initial='1461 days',    # ~4 years for initial training
    period='168 days',      # how much data to add to the training data after every iteration
    horizon='365 days',     # forecast out 1 year
)

# Back-transform the predictions and true values from log to original scale
df_cv['yhat_orig'] = np.exp(df_cv['yhat'])
df_cv['y_orig'] = np.exp(df_cv['y'])

rmse_global = np.sqrt(mean_squared_error(df_cv['y_orig'], df_cv['yhat_orig']))
mae_global = mean_absolute_error(df_cv['y_orig'], df_cv['yhat_orig'])
mape_global = np.mean(np.abs((df_cv['y_orig'] - df_cv['yhat_orig']) / df_cv['y_orig'])) * 100

print(f"RMSE : {rmse_global:.2f} million GP")
print(f"MAE  : {mae_global:.2f} million GP")
print(f"MAPE : {mape_global:.2f}%")

## 📊 Model Performance Summary 

These results suggest that the model predicts within approximately 1 million GP of the true price on average, and is off by less than 19% of the actual value in most cases.

Given that bond prices ranged from 2M in 2015 to over 14M in 2025, these are **strong results** — especially considering:
- The long forecast horizon (365 days)
- The volatility of the OSRS in-game economy
- The lack of external features (e.g. patch updates, player count, or event data)

The model captures the long-term trend and seasonal patterns effectively, while also expressing reasonable uncertainty during volatile periods. While there is still room for improvement (e.g. incorporating external regressors), this baseline model performs well and demonstrates reliable forecasting ability.

    Planned update: Once I acquire player count data, I plan to retrain this model with it as an external regressor to assess its impact on forecast accuracy.

## 📈 Future forecast
Train the model using the full historical dataset to forecast bond prices 1 year into the future

In [ ]:
df_prophet = df.reset_index().rename(columns={'Date': 'ds', 'Price (M)': 'y'})
df_prophet['y_log'] = np.log(df_prophet['y'])

full_df = df_prophet[['ds', 'y_log']].rename(columns={'y_log': 'y'})

final_model = Prophet()
final_model.fit(full_df)

# Make future dataframe: forecast 365 days ahead
future = final_model.make_future_dataframe(periods=365)
forecast = final_model.predict(future)

# Transform back to original price
forecast['yhat_original'] = np.exp(forecast['yhat'])
forecast['yhat_upper_original'] = np.exp(forecast['yhat_upper'])
forecast['yhat_lower_original'] = np.exp(forecast['yhat_lower'])

# Filter from 2020 onwards for plotting
forecast_recent = forecast[forecast['ds'] >= '2020-01-01']
actual_recent = df_prophet[df_prophet['ds'] >= '2020-01-01']

plt.figure(figsize=(14, 7))
plt.plot(actual_recent['ds'], actual_recent['y'], 'b-', label='Actual Price', linewidth=1.5, alpha=0.6)
plt.plot(forecast_recent['ds'], forecast_recent['yhat_original'], 'm-', label='Forecast (All)', linewidth=1.2)
plt.axvline(x=df_prophet['ds'].max(), color='gray', linestyle='--', alpha=0.6, label='Forecast Start',linewidth=1)

plt.fill_between(
    forecast['ds'],
    forecast['yhat_lower_original'],
    forecast['yhat_upper_original'],
    where=forecast['ds'] > df_prophet['ds'].max(),
    color='salmon', alpha=0.3, label='Uncertainty (Future)'
)


plt.title('Forecasting OSRS Bond Prices (Next 1 Year)', fontsize=16)

plt.xlabel('Date')
plt.ylabel('Price (Millions of GP)')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()